# Foursquare

Send a request to Foursquare with a small radius (1000m) for all the bike stations in your city of choice. 

In [1]:
import requests
import time
import logging
import pandas as pd
logging.basicConfig(level=logging.INFO)

# API Keys
FOURSQUARE_API_KEY = "fsq31DSn1KRJPdC/gOri5ab7GA2IX4dWhKTCH/nfGOjXPtg="
YELP_API_KEY = "2vJZU9b6J9f4qUjmHONKfQYb9LlWrgmGdSF7EhzzsNtBEKDiaje2fCnJEN7ADuzcmGAjl2pZOCMBqEW_1NGIBI4tNksgQ9AE1Y_4GzA2BG1DqHeenXh8pAo0GVUAaHYx"

# Define headers
foursquare_headers = {
    "Authorization": f"Bearer {FOURSQUARE_API_KEY}"
}

yelp_headers = {
    "Authorization": f"Bearer {YELP_API_KEY}"
}


In [3]:
# Load the CSV file containing London bike stations
london_bike_stations_df = pd.read_csv('london_bike_stations.csv')  # use single underscore

# Convert to list of dictionaries
london_stations = london_bike_stations_df.to_dict(orient='records')

# Quick check
print(f"Loaded {len(london_stations)} stations")
print(london_stations[0])

Loaded 797 stations
{'name': '300073 - Prince of Wales Drive, Battersea Park', 'latitude': 51.47515398, 'longitude': -0.159169801, 'free_bikes': 13, 'empty_slots': 5, 'total_bikes': 18}


In [4]:
foursquare_headers = {
    "Authorization": f"Bearer {FOURSQUARE_API_KEY}"
}
# Define foursquare search function with retry logic
def search_foursquare(lat, lon, radius=1000, limit=20, query=None, category_ids=None, retries=3):
    url = "https://api.foursquare.com/v3/places/search"
    params = {
        "ll": f"{lat},{lon}",
        "radius": radius,
        "limit": limit
    }
    if query:
        params["query"] = query
    if category_ids:
        params["categories"] = ','.join(map(str, category_ids))

    for _ in range(retries):
        response = requests.get(url, headers=foursquare_headers, params=params)
        if response.status_code == 200:
            return response.json().get("results", [])
        else:
            logging.warning(f"Error {response.status_code}: {response.text}. Retrying...")
            time.sleep(2)
    return []

# Category IDs for transport-related places
transport_category_ids = [
    19042,  # Bus Station
    19043,  # Bus Stop
    19066,  # Train Station
    19067,  # Bus Line
    19068,  # Taxi Service
    19063   # Light Rail Station
]

# Store all results
all_results = []

# Loop through bike stations and query foursquare
for station in london_stations:
    lat = station['latitude']
    lon = station['longitude']
    station_name = station['name']

    # Restaurants near station
    restaurant_results = search_foursquare(lat, lon, radius=1000, limit=20, query="restaurant")
    for place in restaurant_results:
        poi_name = place.get("name", "Unknown")
        poi_latitude = place.get("geocodes", {}).get("main", {}).get("latitude")
        poi_longitude = place.get("geocodes", {}).get("main", {}).get("longitude")
        categories = [cat["name"] for cat in place.get("categories", [])] or ["No categories"]
        address = place.get("location", {}).get("formatted_address", "No address provided")

        all_results.append({
            "station": station_name,
            "poi_type": "restaurant",
            "poi_name": poi_name,
            "poi_latitude": poi_latitude,
            "poi_longitude": poi_longitude,
            "categories": categories,
            "distance": place.get("distance", "N/A"),
            "address": address
        })

    # Transport POIs near station
    transport_results = search_foursquare(lat, lon, radius=1000, limit=20, category_ids=transport_category_ids)
    for place in transport_results:
        poi_name = place.get("name", "Unknown")
        poi_latitude = place.get("geocodes", {}).get("main", {}).get("latitude")
        poi_longitude = place.get("geocodes", {}).get("main", {}).get("longitude")
        categories = [cat["name"] for cat in place.get("categories", [])] or ["No categories"]
        address = place.get("location", {}).get("formatted_address", "No address provided")

        all_results.append({
            "station": station_name,
            "poi_type": "transport",
            "poi_name": poi_name,
            "poi_latitude": poi_latitude,
            "poi_longitude": poi_longitude,
            "categories": categories,
            "distance": place.get("distance", "N/A"),
            "address": address
        })

# Save results
df_results = pd.DataFrame(all_results)
df_results.to_csv("london_stations_stations_pois_fs.csv", index=False)
logging.info("Saved to london_stations_pois_fs.csv")

KeyboardInterrupt: 

Send a request to Yelp with a small radius (1000m) for all the bike stations in your city of choice. 

In [7]:
# Set up logging
logging.basicConfig(level=logging.INFO)

yelp_headers = {
    "Authorization": f"Bearer {YELP_API_KEY}"

}

# Function to query Yelp API and get business results
def query_yelp(lat, lon, categories="restaurants", radius=1000, limit=50):
    """Function to query Yelp API and get business results"""
    url = "https://api.yelp.com/v3/businesses/search"
    params = {
        "latitude": lat,
        "longitude": lon,
        "radius": radius,
        "categories": categories,
        "limit": limit
    }
    try:
        response = requests.get(url, headers=headers, params=params)
        response.raise_for_status()  # Raise an exception for HTTP errors
        return response.json().get("businesses", [])
    except requests.exceptions.RequestException as e:
        logging.error(f"Request failed: {e}")
        return []

# Load your London bike stations
london_stations_df = pd.read_csv('london_bike_stations.csv')

# Prepare list to collect venue data
all_venue_data = []

# Loop through each station and collect Yelp data
for index, station in london_stations_df.iterrows():
    lat = station['latitude']
    lon = station['longitude']
    station_name = station['name']  # Adjust to your DataFrame column!
    logging.info(f"Querying for POIs near: {station_name}")

    results = query_yelp(lat, lon, radius=5000)  # Increased radius!

    logging.info(f"Found {len(results)} businesses for station {station_name}")

    if not results:
        continue

    for business in results:
        venue_details = {
            "station_name": station_name,
            "latitude": lat,
            "longitude": lon,
            "business_name": business.get("name"),
            "rating": business.get("rating"),
            "review_count": business.get("review_count"),
            "distance_m": business.get("distance"),
            "business_latitude": business["coordinates"].get("latitude"),
            "business_longitude": business["coordinates"].get("longitude"),
            "categories": ", ".join([cat["title"] for cat in business.get("categories", [])]),
            "address": business["location"].get("address1", ""),
            "city": business["location"].get("city", ""),
            "state": business["location"].get("state", ""),
            "postal_code": business["location"].get("zip_code", ""),
        }
        all_venue_data.append(venue_details)

    # Respect API rate limits
    time.sleep(1)

# Log the collected data
logging.info(f"Collected data for {len(all_venue_data)} venues.")

# Convert to DataFrame and save
df_venues = pd.DataFrame(all_venue_data)
df_venues.to_csv("london_bike_stations_pois_yelp.csv", index=False)
logging.info("Data saved to london_bike_stations_pois_yelp.csv")

INFO:root:Querying for POIs near: 300073 - Prince of Wales Drive, Battersea Park


NameError: name 'headers' is not defined

# YELP

Parse through the response to get the POI (such as restaurants, bars, etc) details you want (ratings, name, location, etc)

In [9]:
all_venue_data = []

results = response.json().get("businesses", [])

for business in results:
    poi = {
        "business_name": business.get("name"),
        "rating": business.get("rating"),
        "review_count": business.get("review_count"),
        "price": business.get("price", "N/A"),
        "distance_m": business.get("distance"),
        "latitude": business["coordinates"].get("latitude"),
        "longitude": business["coordinates"].get("longitude"),
        "categories": ", ".join([cat["title"] for cat in business.get("categories", [])]),
        "address": business["location"].get("address1", ""),
        "city": business["location"].get("city", ""),
        "state": business["location"].get("state", ""),
        "postal_code": business["location"].get("zip_code", ""),
        "phone": business.get("display_phone", "N/A"),
        "url": business.get("url")
    }

    # Save each point of interest
    all_venue_data.append(poi)

NameError: name 'response' is not defined

Put your parsed results into a DataFrame

In [10]:
# Log the collected data 
logging.info(f"Collected data for {len(all_venue_data)} venues.")

# Convert list of venues to dataframe
df_venues = pd.DataFrame(all_venue_data)

# Save the data to a csv file 
df_venues.to_csv("london_bike_stations_pois_yelp.csv", index=False)
logging.info("Data saved to london_bike_stations_pois_yelp.csv")

INFO:root:Collected data for 0 venues.
INFO:root:Data saved to london_bike_stations_pois_yelp.csv


# Comparing Results

Which API provided you with more complete data? Provide an explanation. 

Get the top 10 restaurants according to their rating

In [36]:
# Log the collected data 
logging.info(f"Collected data for {len(all_venue_data)} venues.")

# Convert list of venues to dataFrame
df_venues = pd.DataFrame(all_venue_data)

# Optionally, save the data to a csv file for later use
df_venues.to_csv("london_bike_stations_pois_yelp.csv", index=False)
logging.info("Data saved to london_bike_stations_pois_yelp.csv")

INFO:root:Collected data for 10400 venues.
INFO:root:Data saved to london_bike_stations_pois_yelp.csv


In [33]:
import requests

headers = {
    "Authorization": f"Bearer {YELP_API_KEY}"
}

latitude = 51.5074  # Central London
longitude = -0.1278
radius = 5000  # 5km

params = {
    "latitude": latitude,
    "longitude": longitude,
    "radius": radius,
    "sort_by": "rating",
    "limit": 50
}

response = requests.get("https://api.yelp.com/v3/businesses/search", headers=headers, params=params)
data = response.json()

print("Total businesses found:", data.get('total', 0))

for business in data.get('businesses', []):
    print(business['name'], "-", business['rating'])

Total businesses found: 15600
Via Emilia by FOOD ROOTS - 5.0
Moira Restaurant - 5.0
Pleasant Lady Jian Bing Trading Stall - 5.0
Red Bar - 5.0
Barbican Estate - 5.0
Cafe Van Gogh - 5.0
Murder Inc - 5.0
Nepa Coffee & Food - 5.0
Crol & co - 5.0
Pure Indian Cooking - 5.0
Skewers - 5.0
Cous Cous Darna - 5.0
Il Piccolo Diavolo - 5.0
Kazu - 5.0
Good And Proper Tea - 5.0
Rainforest Creations - 5.0
Chokhi Dhani - 5.0
Second Shot Coffee - 5.0
The Clifton - 5.0
Nama Café and Cake House - 5.0
Hot Chip - 5.0
Terra Rossa - 5.0
Scarpetta - 5.0
39 Steps Coffee - 5.0
Sir Sydney Smith - 5.0
Pide - 5.0
Kibele - 5.0
Holloway Best Kebab - 5.0
Merchant House - 5.0
Jean-Georges - 5.0
Trade - 5.0
Delicatessen Restaurant - 5.0
Patara Restaurant - 5.0
The Black Cab Coffee Co - 5.0
Oi Vita Pizzeria - 5.0
Da Terra - 5.0
Chojo - 5.0
Vinarius Wine Shop and Bar - 5.0
Noble Espresso - 5.0
Sinuhe Restaurant - 5.0
Beer + Burger - 5.0
Gastronhome - 5.0
Boki - 5.0
Istanbul Mangal - 5.0
The Fumoir Bar - 5.0
African Volcan